# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

<b>Признаки</b>

<br>DateCrawled — дата скачивания анкеты из базы
<br>VehicleType — тип автомобильного кузова
<br>RegistrationYear — год регистрации автомобиля
<br>Gearbox — тип коробки передач
<br>Power — мощность (л. с.)
<br>Model — модель автомобиля
<br>Kilometer — пробег (км)
<br>RegistrationMonth — месяц регистрации автомобиля
<br>FuelType — тип топлива
<br>Brand — марка автомобиля
<br>Repaired — была машина в ремонте или нет
<br>DateCreated — дата создания анкеты
<br>NumberOfPictures — количество фотографий автомобиля
<br>PostalCode — почтовый индекс владельца анкеты (пользователя)
<br>LastSeen — дата последней активности пользователя

<br><b>Целевой признак</b>
<br>Price — цена (евро)

## Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import datetime as DT
import matplotlib.pyplot as plt

from sklearn.dummy import DummyClassifier
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder


In [2]:
data = pd.read_csv('/datasets/autos.csv')

In [3]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
data.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Много пропущенных значений:

<br>VehicleType — тип автомобильного кузова
<br>Gearbox — тип коробки передач
<br>Model — модель автомобиля
<br>FuelType — тип топлива
<br>Repaired — была машина в ремонте или нет

In [6]:
data['VehicleType'].value_counts()

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: VehicleType, dtype: int64

In [7]:
data['Gearbox'].value_counts()

manual    268251
auto       66285
Name: Gearbox, dtype: int64

In [8]:
data['Model'].value_counts()

golf                  29232
other                 24421
3er                   19761
polo                  13066
corsa                 12570
                      ...  
i3                        8
serie_3                   4
rangerover                4
range_rover_evoque        2
serie_1                   2
Name: Model, Length: 250, dtype: int64

In [9]:
data['FuelType'].value_counts()

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

In [10]:
data['Repaired'].value_counts() 

no     247161
yes     36054
Name: Repaired, dtype: int64

Тут самый часто встречаемый вариант "no", попробуем заменить пропуски на него.


In [11]:
data['Repaired'] = data['Repaired'].fillna("no")

In [12]:
data.duplicated().sum()

5

Я бы предложила дропнуть все пропущенные значения и дубликаты

In [13]:
data = data.dropna()
data = data.drop_duplicates()

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284121 entries, 2 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        284121 non-null  object
 1   Price              284121 non-null  int64 
 2   VehicleType        284121 non-null  object
 3   RegistrationYear   284121 non-null  int64 
 4   Gearbox            284121 non-null  object
 5   Power              284121 non-null  int64 
 6   Model              284121 non-null  object
 7   Kilometer          284121 non-null  int64 
 8   RegistrationMonth  284121 non-null  int64 
 9   FuelType           284121 non-null  object
 10  Brand              284121 non-null  object
 11  Repaired           284121 non-null  object
 12  DateCreated        284121 non-null  object
 13  NumberOfPictures   284121 non-null  int64 
 14  PostalCode         284121 non-null  int64 
 15  LastSeen           284121 non-null  object
dtypes: int64(7), object(

In [15]:
data.isna().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
Repaired             0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

Посмотрим данные с датами

In [16]:
print(data['DateCrawled'].min())
print(data['DateCrawled'].max())

2016-03-05 14:06:22
2016-04-07 14:36:58


In [17]:
print(data['DateCreated'].min())
print(data['DateCreated'].max())

2015-03-20 00:00:00
2016-04-07 00:00:00


In [18]:
print(data['LastSeen'].min())
print(data['LastSeen'].max())

2016-03-05 14:15:16
2016-04-07 14:58:51


In [19]:
data['RegistrationMonth'].value_counts()

3     29692
6     26596
4     24856
5     24627
7     23126
10    22355
12    20670
9     20547
11    20500
1     19609
8     19218
2     18302
0     14023
Name: RegistrationMonth, dtype: int64

Месяц 0 предлагаю дропнуть

In [20]:
#data = data[data['RegistrationMonth']> 0]
data = data.drop(['RegistrationMonth'], axis=1)

In [21]:
data['RegistrationYear'].value_counts()

1999    19675
2006    18170
2003    17901
2005    17877
2004    17852
        ...  
1931        1
1928        1
1927        1
1947        1
1919        1
Name: RegistrationYear, Length: 88, dtype: int64

In [22]:
print(data['RegistrationYear'].min())
print(data['RegistrationYear'].max())

1910
2018


Предлагаю взять данные хотя бы от 2000 года.

In [23]:
data = data[data['RegistrationYear']> 1999]
data = data[data['RegistrationYear']< 2017]
data['RegistrationYear'].value_counts()

2006    18170
2003    17901
2005    17877
2004    17852
2001    17813
2000    17247
2002    17080
2007    15704
2008    14215
2009    13530
2010    10083
2011     9374
2012     6798
2013     3891
2014     2374
2015     1035
2016      149
Name: RegistrationYear, dtype: int64

In [24]:
data_to_ohe = data.drop(['DateCrawled', 'LastSeen', 'DateCreated', 'PostalCode'], axis=1)
data_to_ohe.isna().sum()

Price               0
VehicleType         0
RegistrationYear    0
Gearbox             0
Power               0
Model               0
Kilometer           0
FuelType            0
Brand               0
Repaired            0
NumberOfPictures    0
dtype: int64

## Обучение моделей

In [28]:
df_train, df_valid = train_test_split(data_to_ohe, test_size=0.20, random_state=12345) 

In [29]:
df_features_train = df_train.drop(['Price'], axis=1)
df_target_train = df_train['Price']
features_valid = df_valid.drop(['Price'], axis=1)
target_valid = df_valid['Price']

In [30]:
features_train, features_test,target_train, target_test  = train_test_split(df_features_train, df_target_train, 
                                                                            test_size = 0.25, random_state=12345)

In [31]:
features_train_ohe = pd.get_dummies(features_train, drop_first=True)
features_valid_ohe = pd.get_dummies(features_valid, drop_first=True)
features_test_ohe = pd.get_dummies(features_test, drop_first=True)
# после прямого кодирования кол-во признаков в выборках будет отличаться
# поэтому стоит удалить лишние признаки в каждой выборке

# Найдем общие признаки между обучающей, валидационной и тестовой выборками
common_columns = set(features_train_ohe.columns) & set(features_valid_ohe.columns) & set(features_test_ohe.columns)

# Оставим в каждой выборке только общие признаки
features_train_ohe = features_train_ohe[list(common_columns)]
features_valid_ohe = features_valid_ohe[list(common_columns)]
features_test_ohe = features_test_ohe[list(common_columns)]

In [32]:
print('\n','Размер датасета',data_to_ohe.shape,'\n', 'Признаки тренеровочной выборки',features_train.shape,'\n','Таргет тренировочной выборки', target_train.shape,'\n','Признаки валид. выборки', features_valid.shape,'\n','Таргет валид. выборки', target_valid.shape,'\n','Признаки тестовой', features_test.shape,'\n','Таргет тестовой', target_test.shape)



 Размер датасета (201093, 11) 
 Признаки тренеровочной выборки (120655, 10) 
 Таргет тренировочной выборки (120655,) 
 Признаки валид. выборки (40219, 10) 
 Таргет валид. выборки (40219,) 
 Признаки тестовой (40219, 10) 
 Таргет тестовой (40219,)


### GBM

Хотелось бы сразу написать комментарий почему я не стала добавлять грид сёрч по gbm. Потому что там время на подбор параметров у меня уходило почти 2-3 часа, не хочу замедлять отработку кода своего ноутбука, поэтому я вручную просто подобрала параметры для модели.

In [33]:
gbm = lgb.LGBMRegressor(random_state=12345,max_depth = 10, n_estimators = 500, learning_rate=0.1)

In [34]:
%%time
gbm.fit(features_train_ohe, target_train)

CPU times: user 13.5 s, sys: 179 ms, total: 13.7 s
Wall time: 13.9 s


LGBMRegressor(max_depth=10, n_estimators=500, random_state=12345)

In [35]:
%%time
predict = gbm.predict(features_valid_ohe)

CPU times: user 1.37 s, sys: 56.3 ms, total: 1.42 s
Wall time: 1.41 s


In [36]:
mse = mean_squared_error(target_valid, predict)
rmse = mse**(0.5)
print("RMSE: %.2f" % rmse) 

RMSE: 1695.22


### Catboost

In [3]:
catboost = CatBoostRegressor()
params_cat = {'learning_rate': [0.03, 0.1, 0.2],
        'depth': [4, 6, 10],
        'loss_function': ['RMSE']}

grid_catboost = catboost.grid_search(params_cat,
                                       X=features_train_ohe,
                                       y=target_train,
                                       plot=True)

NameError: name 'CatBoostRegressor' is not defined

In [38]:
grid_catboost["params"]

{'depth': 10, 'learning_rate': 0.2}

Замерим время обучения с подобранными параметрами.

In [39]:
%%time
cat = CatBoostRegressor(depth=10, learning_rate=0.2)
cat.fit(features_train_ohe,target_train)

0:	learn: 4101.7892994	total: 41.1ms	remaining: 41s
1:	learn: 3602.7947641	total: 76.7ms	remaining: 38.3s
2:	learn: 3208.8514754	total: 119ms	remaining: 39.4s
3:	learn: 2904.7913350	total: 154ms	remaining: 38.2s
4:	learn: 2688.9645443	total: 188ms	remaining: 37.4s
5:	learn: 2508.5826306	total: 236ms	remaining: 39s
6:	learn: 2375.1547334	total: 271ms	remaining: 38.4s
7:	learn: 2277.3322156	total: 313ms	remaining: 38.8s
8:	learn: 2202.3515827	total: 350ms	remaining: 38.5s
9:	learn: 2142.4230577	total: 386ms	remaining: 38.2s
10:	learn: 2090.0910778	total: 430ms	remaining: 38.7s
11:	learn: 2052.4742381	total: 468ms	remaining: 38.6s
12:	learn: 2019.9557966	total: 514ms	remaining: 39s
13:	learn: 1995.5642245	total: 549ms	remaining: 38.7s
14:	learn: 1976.5826869	total: 583ms	remaining: 38.3s
15:	learn: 1963.0218188	total: 629ms	remaining: 38.7s
16:	learn: 1944.9961047	total: 663ms	remaining: 38.4s
17:	learn: 1930.5889864	total: 710ms	remaining: 38.7s
18:	learn: 1920.1499254	total: 748ms	remai

In [40]:
%%time
pred = cat.predict(features_valid_ohe) 

CPU times: user 210 ms, sys: 0 ns, total: 210 ms
Wall time: 212 ms


In [41]:
mse = mean_squared_error(target_valid, pred)
rmse = mse**(0.5)
print("RMSE: %.2f" % rmse) 

RMSE: 1647.04


### Random Forest

In [42]:
data_to_ohe.columns

Index(['Price', 'VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
       'Kilometer', 'FuelType', 'Brand', 'Repaired', 'NumberOfPictures'],
      dtype='object')

In [43]:
# порядковое кодирование (для "деревянных моделей")
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=9999)
cat_columns = ['VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
       'Kilometer', 'FuelType', 'Brand', 'Repaired']
encoder.fit(features_train[cat_columns])

features_train_ord = features_train.copy()
features_valid_ord = features_valid.copy()
features_test_ord = features_test.copy()

features_train_ord[cat_columns] = encoder.transform(features_train_ord[cat_columns])
features_valid_ord[cat_columns] = encoder.transform(features_valid_ord[cat_columns])
features_test_ord[cat_columns] = encoder.transform(features_test_ord[cat_columns])

In [44]:
%%time
rfr = RandomForestRegressor(random_state=12345,n_estimators = 100,n_jobs=-1)
rfr.fit(features_train_ord, target_train)

CPU times: user 23 s, sys: 333 ms, total: 23.3 s
Wall time: 23.3 s


RandomForestRegressor(n_jobs=-1, random_state=12345)

In [45]:
%%time
pred = rfr.predict(features_valid_ord) 

CPU times: user 1.29 s, sys: 466 µs, total: 1.29 s
Wall time: 1.3 s


In [46]:
mse = mean_squared_error(target_valid, pred)
rmse = mse**(0.5)
print("RMSE: %.2f" % rmse) 

RMSE: 1766.76


## Анализ моделей

In [52]:
result_data = [["GBM", "13.9 s", " 1695.22"],
         ["Catboost", "39.9 s", "1647.04"],
         ["RF", "23.3", "1766.76"]]
result= pd.DataFrame(result_data, columns=["Model","Wall time", "RMSE"])
result = result.set_index('Model')

In [53]:
result

,Wall time,RMSE
Model,,
GBM,13.9 s,1695.22
Catboost,39.9 s,1647.04
RF,23.3,1766.76


По результатам работы победила Catboost.

## Тестирование лучшей модели

In [49]:
pred = cat.predict(features_test_ohe) 
mse = mean_squared_error(target_test, pred)
rmse = mse**(0.5)
print("RMSE: %.2f" % rmse) 

RMSE: 1645.42


# Общий вывод

В данных много пропущенных значений, почти 30% от полного датасета. Есть аномалии: 0 месяца регистрации, 1930 года и т.д. В данной работе я выбрала дропнуть большинство пропущенных значений. Удалила неинформативные столбцы и дубликаты.
    Самый лучший результат по скорости и качеству обучения показала - Catboost.